<a href="https://colab.research.google.com/github/UEPP40/PUM/blob/Debere/Bigdatatest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark==1.3.0
!pip install -q pyspark==3.5.1
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext


findspark.init("spark-3.5.1-bin-hadoop3")
sc = pyspark.SparkContext('local[*]')
spark = SparkSession.builder.appName('abc').getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
!free -mh # Komenda na sprawdzenie dostępnej pamięci RAM
import pyspark.sql.functions as f
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, TimestampType


               total        used        free      shared  buff/cache   available
Mem:            12Gi       1.0Gi       1.7Gi       1.0Mi         9Gi        11Gi
Swap:             0B          0B          0B


In [3]:
schema_for_clients = StructType([
    StructField('PREFIX', StringType()),
    StructField('country_code', StringType()),
    StructField('region_id', IntegerType()),
    StructField('first_activity', TimestampType()),
    StructField('last_activity', TimestampType())
])

In [25]:
import pandas as pd

file=pd.read_excel('Zadanie stażowe - Junior Business Analyst.xlsx')
clients=spark.createDataFrame(file, schema=schema_for_clients)

In [26]:
clients.printSchema()

root
 |-- PREFIX: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- region_id: integer (nullable = true)
 |-- first_activity: timestamp (nullable = true)
 |-- last_activity: timestamp (nullable = true)



In [27]:
clients.show()

+--------------------+------------+---------+-------------------+-------------------+
|              PREFIX|country_code|region_id|     first_activity|      last_activity|
+--------------------+------------+---------+-------------------+-------------------+
|125e7c32c75x1470d378|          PL|        6|2024-04-05 18:48:58|2024-04-05 18:48:58|
|137178d76e7x697d40d9|          PL|       14|2024-04-13 13:25:59|2024-04-13 13:25:59|
|13951d78ccax69d403c1|          PL|        8|2023-12-10 11:25:57|2023-12-10 11:25:57|
|14fc1cfc985x33fe98a6|          PL|       15|2023-09-20 18:09:54|2023-09-20 18:09:54|
|14fd495fbd6x4dd7aae8|          PL|        1|2023-09-17 21:06:51|2023-10-02 15:40:41|
|14fd4c69808x72b705a7|          PL|       17|2023-09-16 08:53:05|2023-09-16 08:53:05|
| 14fd4c7655bx4652c86|          PL|       13|2023-09-30 18:03:40|2023-09-30 18:03:40|
|14fd4c7c56dx4a0d8bd4|          PL|        4|2023-09-16 08:12:08|2023-09-16 08:16:50|
|14fd4cb2508x439d287f|          PL|       13|2024-01-2

In [28]:
clients.createOrReplaceTempView('clients')

In [29]:
liczba_klientow=spark.sql("""SELECT count(*) AS Liczba_klientow  from clients""")
liczba_klientow.show()

+---------------+
|Liczba_klientow|
+---------------+
|           5000|
+---------------+



In [30]:
zad11=spark.sql("""SELECT region_id, COUNT(*) as liczba_uzytkownikow
FROM clients
GROUP BY region_id
order by region_id""")
zad11.show()

+---------+-------------------+
|region_id|liczba_uzytkownikow|
+---------+-------------------+
|        1|                573|
|        2|                684|
|        3|                387|
|        4|                366|
|        5|                332|
|        6|                575|
|        7|                341|
|        8|                280|
|        9|                152|
|       11|                236|
|       12|                 98|
|       13|                167|
|       14|                181|
|       15|                303|
|       17|                197|
|       18|                128|
+---------+-------------------+



In [31]:
zad11b=spark.sql("""WITH RegionCount AS (
    SELECT region_id, COUNT(*) as liczba_uzytkownikow
FROM clients
GROUP BY region_id
)
SELECT region_id, liczba_uzytkownikow
FROM RegionCount
WHERE liczba_uzytkownikow = (SELECT MAX(liczba_uzytkownikow) FROM RegionCount) or liczba_uzytkownikow = (SELECT MIN(liczba_uzytkownikow) FROM RegionCount)""")
zad11b.show()

+---------+-------------------+
|region_id|liczba_uzytkownikow|
+---------+-------------------+
|       12|                 98|
|        2|                684|
+---------+-------------------+



In [32]:
zad12=spark.sql("""Select region_id, count(*) AS Jednorazowi_klienci from clients where first_activity=last_activity group by region_id order by region_id""")
zad12.show() #wizualizacja tabela z powyżej

+---------+-------------------+
|region_id|Jednorazowi_klienci|
+---------+-------------------+
|        1|                248|
|        2|                285|
|        3|                162|
|        4|                174|
|        5|                144|
|        6|                261|
|        7|                147|
|        8|                127|
|        9|                 63|
|       11|                113|
|       12|                 46|
|       13|                 55|
|       14|                 80|
|       15|                137|
|       17|                 94|
|       18|                 66|
+---------+-------------------+



In [33]:
zad12b=spark.sql("""Select count(*) AS Jednorazowi_klienci from clients where first_activity=last_activity""")
zad12b.show()

+-------------------+
|Jednorazowi_klienci|
+-------------------+
|               2202|
+-------------------+



In [34]:
zad13=spark.sql("""Select * from clients where first_activity=last_activity and first_activity<'2024-01-01 00:00:00'""")
zad13.show()

+--------------------+------------+---------+-------------------+-------------------+
|              PREFIX|country_code|region_id|     first_activity|      last_activity|
+--------------------+------------+---------+-------------------+-------------------+
|13951d78ccax69d403c1|          PL|        8|2023-12-10 11:25:57|2023-12-10 11:25:57|
|14fc1cfc985x33fe98a6|          PL|       15|2023-09-20 18:09:54|2023-09-20 18:09:54|
|14fd4c69808x72b705a7|          PL|       17|2023-09-16 08:53:05|2023-09-16 08:53:05|
| 14fd4c7655bx4652c86|          PL|       13|2023-09-30 18:03:40|2023-09-30 18:03:40|
|14fd4ee99c8x4a1a6e14|          PL|        6|2023-09-16 10:20:09|2023-09-16 10:20:09|
|14fd4f14e61x72034f8b|          PL|        2|2023-10-08 21:17:23|2023-10-08 21:17:23|
|14fd4faf682x49352408|          PL|       11|2023-09-18 12:18:55|2023-09-18 12:18:55|
|14fd501522dx516e4dde|          PL|       18|2023-11-30 11:42:27|2023-11-30 11:42:27|
|14fd5035024x72146fcf|          PL|       17|2023-10-1

In [14]:
zad13=spark.sql("""Select * from clients where first_activity=last_activity and first_activity BETWEEN '2023-11-25 00:00:00' and '2024-01-01 00:00:00'""")
zad13.show(n=1000, truncate=False) #święta

+--------------------+------------+---------+-------------------+-------------------+
|PREFIX              |country_code|region_id|first_activity     |last_activity      |
+--------------------+------------+---------+-------------------+-------------------+
|13951d78ccax69d403c1|PL          |8        |2023-12-10 11:25:57|2023-12-10 11:25:57|
|14fd501522dx516e4dde|PL          |18       |2023-11-30 11:42:27|2023-11-30 11:42:27|
|14fd59bac6bx63d6ae7f|PL          |2        |2023-12-30 14:50:59|2023-12-30 14:50:59|
|14fd5b8b8d8x7d765624|PL          |5        |2023-12-31 11:22:50|2023-12-31 11:22:50|
|14fd65b1ba4x29491e48|PL          |7        |2023-12-11 18:50:52|2023-12-11 18:50:52|
|14fd6e80e05x5f21ddca|PL          |6        |2023-11-28 20:57:59|2023-11-28 20:57:59|
|14fd73e98bax5e919a10|PL          |13       |2023-11-26 13:55:15|2023-11-26 13:55:15|
|14fd75d6f4bx567cb7b5|PL          |4        |2023-12-12 21:06:58|2023-12-12 21:06:58|
|14fda9d3499x34de6e89|PL          |8        |2023-12-2

In [35]:
with pd.ExcelWriter('answer.xlsx') as writer:
     zad11.toPandas().to_excel(writer, sheet_name='zad1.1')
     zad11b.toPandas().to_excel(writer, sheet_name='zad1.1b')
     zad12.toPandas().to_excel(writer, sheet_name='zad1.2')
     zad12b.toPandas().to_excel(writer, sheet_name='zad1.2b')
     zad13.toPandas().to_excel(writer, sheet_name='zad1.3')
#zad11 = zad11.toPandas()
#zad11.to_excel('wynik.xlsx', index=False)

 **Zadanie 2**

In [ ]:
dane2 = pd.read_excel('Zadanie stażowe - Junior Business Analyst.xlsx', sheet_name=['Dane_2'])

In [ ]:
type(dane2)

dict